<a href="https://colab.research.google.com/github/Azureuse/colab/blob/main/portfolio_selection_optimization/portfolio_selection_optimization_gcl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Selection Optimization
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 


## Model Formulation
### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $\delta$: n-element vector measuring the change in price for each stock
- $\sigma$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the porfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):

\begin{equation}
e \cdot x = 1
\end{equation}


Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:

\begin{equation}
\delta \cdot x = r
\end{equation}

## Python Implementation
### Stock data
Use [yfinance](https://pypi.org/project/yfinance/) library to get the latest 2 years of _actual stock data_ from the 20 most profitable US companies, [according to Wikipedia in April 2021](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit).

In [65]:
%pip install gurobipy yfinance pytickersymbols

In [1]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
stocks = list(stock_data.get_sp_100_nyc_yahoo_tickers())
len(stocks)

104

In [32]:
import yfinance as yf

# stocks = ['BRK-A', 'AAPL', 'MSFT', 'JPM', 'GOOG', 'BAC', 'INTC', 'WFC',
#           'C', 'VZ', 'FB', 'PFE', 'JNJ', 'WMT', 'XOM',
#           'FNMA', 'T', 'UNH', 'CMCSA', 'V' ]

data = yf.download(stocks, period='2y')
data.shape

[*********************100%***********************]  104 of 104 completed


(505, 624)

### Compute Greeks
Using the downloaded stock data, find the delta (return), sigma (covariance) and standard deviation values for stock prices:

In [59]:
import numpy as np

clean_close_data = data['Close'].dropna(axis=1, how='any')

clean_close_data.shape

(505, 102)

In [74]:
n = 80

closes = np.transpose(np.array(clean_close_data.iloc[300:, :n])) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
delta = np.mean(reldiff, axis=1)            # mean price change
sigma = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation

sigma.shape

(80, 80)

## Minimize risk by solving QP model

In [84]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt

# Create an empty model
m = gp.Model('portfolio')

n = len(delta)

# Add matrix variable for the stocks
# x = m.addMVar(n, vtype='C', lb=0, ub=0.06)
x = m.addMVar(n, vtype='S', lb=0.012, ub=0.06)

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = (x @ sigma @ x) + (0.10 * (x @ x)) - (1.0 * (x @ delta))
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with a constraint
m.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1 rows, 80 columns and 80 nonzeros
Model fingerprint: 0x5002b937
Model has 3240 quadratic objective terms
Variable types: 0 continuous, 0 integer (0 binary)
Semi-Variable types: 80 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 2e-01]
  Bounds range     [1e-02, 6e-02]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 161 rows, 160 columns, 400 nonzeros
Presolved model has 3240 quadratic objective terms
Variable types: 80 continuous, 80 integer (80 binary)
Found heuristic solution: objective 0.0059339

Root relaxation: objective 1.304718e-03, 242 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

## Display minimum risk portfolio using Pandas

In [83]:
import pandas as pd
# minrisk_volatility = sqrt(m.ObjVal)
# minrisk_return = delta @ x.X
# pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
#              index=stocks + ['Volatility', 'Expected Return'],
#              columns=['Minimum Risk Portfolio'])
df_weights = pd.DataFrame(data=x.X, index=stocks[:n], columns=['weight'])
df_weights.sort_values(by=['weight'], ascending=False).head(50)

,weight
VZ,0.021724
COST,0.020985
UNH,0.019554
FDX,0.018460
PYPL,0.017913
QCOM,0.017885
F,0.016757
CAT,0.016644
SBUX,0.016433
DHR,0.015591



## Replace Semicontinuous with Continuous + Binary


In [82]:
# Create an empty model
m2 = gp.Model('portfolio')

n = len(delta)

# Add matrix variable for the stocks
x = m2.addMVar(n, vtype='C')
x_b = m2.addMVar(n, vtype='B')

m2.addConstr(x >= (0.012 * x_b), 'sc_lower')
m2.addConstr(x <= (0.06 * x_b), 'sc_upper')

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = (x @ sigma @ x) + (0.10 * (x @ x)) - (1.0 * (x @ delta))
m2.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with a constraint
m2.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
m2.write('portfolio_selection_optimization2.lp')

# Optimize model to find the minimum risk portfolio
m2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 161 rows, 160 columns and 400 nonzeros
Model fingerprint: 0x324f7ab3
Model has 3240 quadratic objective terms
Variable types: 80 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e-04, 3e-03]
  QObjective range [2e-07, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0053147
Presolve time: 0.00s
Presolved: 161 rows, 160 columns, 400 nonzeros
Presolved model has 3240 quadratic objective terms
Variable types: 80 continuous, 80 integer (80 binary)

Root relaxation: objective 2.481891e-04, 243 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00025    0   80    0.00531 

## Compute the efficient frontier
Solve the QP parametrically to find the lowest risk portfolio for different expected returns.

In [ ]:
# Create an expression representing the expected return for the portfolio
portfolio_return = delta @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target[0].rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

## Plot results
Use the matplot library to plot the optimized solutions, along with the individual stocks:

In [ ]:
import matplotlib.pyplot as plt
#plt.figure(figsize=(10,10))

fig, ax = plt.subplots(figsize=(10,8))

# Plot volatility versus expected return for individual stocks
ax.scatter(x=std, y=delta,
           color='Blue', label='Individual Stocks')
for i, stock in enumerate(stocks):
    ax.annotate(stock, (std[i], delta[i]))

# Plot volatility versus expected return for minimum risk portfolio
ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
            horizontalalignment='right')

# Plot efficient frontier
ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# Format and display the final plot
ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
ax.set_xlabel('Volatility (standard deviation)')
ax.set_ylabel('Expected Return')
ax.legend()
ax.grid()
plt.show()